# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI


# get external variables or functions:
import src.API_key as key


from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback


import json


# Load data to translate
> this should be the drawn concept data set whereby 1 row is 1 drawn concept

In [2]:
print(os.getcwd())
# Get the current working directory
os.chdir("data") 
directory = os.getcwd()

c:\DATEN\PHD\Article_SoftRobotIntervention\Materials\translation process\automatize translation CAM data


In [3]:
# print the current working directory
print(directory)

c:\DATEN\PHD\Article_SoftRobotIntervention\Materials\translation process\automatize translation CAM data\data


In [4]:
# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

['drawnConcepts_translation.xlsx']


In [5]:
# Load the .xlsx file into a DataFrame
df = pd.read_excel(directory + "/drawnConcepts_translation.xlsx")
# Specify the columns to keep
columns_to_keep_text = ['text'] # only the df column 'text' will be kept
columns_to_keep_comment = ['id', 'comment'] # only the df columns 'id' and 'comment' will be kept

# Filter the DataFrame to keep only the specified columns
df_text = df[columns_to_keep_text]
df_comment = df[columns_to_keep_comment]

# prepare the data frames 
# for the text only the unique values are kept
print("Length of the text:")
print(df_text.__len__())
df_text = df_text['text'].str.strip()
df_text = df_text.unique()
# remove "
df_text = [s.replace('"', '') for s in df_text]
print(df_text.__len__())

# for the comment the NaN values are removed
print("Length of the comment:")
print(df_comment.__len__())
df_comment = df_comment.dropna()
df_comment["comment"] = [s.replace('"', '') for s in df_comment["comment"]]
print(df_comment.__len__())

c:\Users\fenn\AppData\Local\R-MINI~1\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Length of the text:
6451
2395
Length of the comment:
6451
1264


In [6]:
print(df_text)
print(df_comment)

['sozialer Assistenzroboter', 'Nachteile', 'Vorteile', 'strukturierte Therapieabläufe', 'günstige Arbeitskräfte', 'Ständige Verfügbarkeit (zeitlich)', 'förderung sozialer Interaktion', 'bei sozialer Isolation', 'bei speziellen Bedürfnissen', 'keine menschliche Nähe', 'verlernen von Zuneigung', 'mangelnde    Empathie', 'weniger Arbeitsplätze', 'Erwartungsänderung an Menschen', 'keine physische Begleitung', 'Kosten für Entwicklung', 'Entwickler:innen erhalten Arbeitsplätze', 'bedingunslose emotionale Begleitung', 'zukünftige Hilfe', 'Wenig Kommunikation', 'Schnelle hygienische Hilfe', '24  Stunden bereit', 'Kann kaputtgehen', 'Besuch', 'Teuer', 'Bedienung', 'Verschwiegen', 'Leise', 'Verführung der Bequemlichkeit', 'Natur der Menschen', "Chekhov's Gun", 'Kein Menschlicher Kontakt', 'Weniger Missbrauch Schwacher', 'Mehr Ressourcen', 'Bessere Technologie Entwicklung', 'Entlastung Pflegepersonal', 'Verrohung des Verhaltens', 'Förderung sozialer Interaktion', 'Unerstützung beim Lernen', 'Unte

# Set up ChatGPT

## Translate written text new

In [7]:
json_schema = {
    "title": "Translation",
    "description": "Dictionary that contains the words and their respective translations.",
    "type": "object",
    "properties": {
        "dictionary": {
            "type": "string",
            "description": 'Dictionary that contains the words as keys and their respective translations as values. Provide the dictionary in the following format: {"word1": "translation1", "word2": "translation2", ...}.',
        },
    },
    "required": ["dictionary"],
}

In [8]:
system_template = """<Context>You are a helpful assistant that translates {language} words to English.</Context>

<Data Structure>The lists "translate" is an array containing {language} words or sentences.</Data Structure>

<Task>Translate the provided array "translate" to English, whereby you return a dictionary with the {language} words as keys and their English translations as values. 
The dictionary should be structured according to the provided JSON schema.
Be careful with spelling errors and do not use capital letters for adjectives.
Repeat this process in case no translations are returned or the dictionary do not follow the JSON schema.</Task>
"""

user_template = """Array "translate": 
{translate}
"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)


result = prompt_template.invoke({"language": "German", "translate":["Hallo", "Guten Tag"]})
print(result)

print("result:", result)
print("result.to_messages():", result.to_messages())

messages=[SystemMessage(content='<Context>You are a helpful assistant that translates German words to English.</Context>\n\n<Data Structure>The lists "translate" is an array containing German words or sentences.</Data Structure>\n\n<Task>Translate the provided array "translate" to English, whereby you return a dictionary with the German words as keys and their English translations as values. \nThe dictionary should be structured according to the provided JSON schema.\nBe careful with spelling errors and do not use capital letters for adjectives.\nRepeat this process in case no translations are returned or the dictionary do not follow the JSON schema.</Task>\n'), HumanMessage(content='Array "translate": \n[\'Hallo\', \'Guten Tag\']\n')]
result: messages=[SystemMessage(content='<Context>You are a helpful assistant that translates German words to English.</Context>\n\n<Data Structure>The lists "translate" is an array containing German words or sentences.</Data Structure>\n\n<Task>Translat

In [9]:
def basic_API_call(
    prompt,
    language,
    translate,
    openai_api_key,
    json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
):

    # prompt = PromptTemplate(template=template)
    seed = 123

    model = ChatOpenAI(model=model_name, openai_api_key=openai_api_key, max_tokens=max_tokens, model_kwargs={"seed": seed}, temperature=0.0)
       
    structured_llm = model.with_structured_output(json_schema, include_raw=True)
    chain = prompt | structured_llm

    with get_openai_callback() as cb:
        response = chain.invoke(
            {"language": language, "translate": translate}
        )
        print(cb)
    
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")
        
    return response

In [12]:
result = basic_API_call(prompt=prompt_template,
    language="German",
    translate=["Hallo", "Guten Tag", "Wie geht es dir?"],
    openai_api_key=key.openai_api_key,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000
)

Tokens Used: 262
	Prompt Tokens: 229
	Completion Tokens: 33
Successful Requests: 1
Total Cost (USD): $0.00164
Total Tokens: 262
Prompt Tokens: 229
Completion Tokens: 33
Total Cost (USD): $0.00164


In [13]:
# Extract the 'parsed' section from the JSON data
parsed_section = result.get('parsed', {})
dictionary = parsed_section.get('dictionary')
    
# Convert the string back to a Python dictionary
print(type(dictionary)) 
dictionary = eval(dictionary)
print(len(dictionary.keys()))
print(dictionary)

<class 'str'>
3
{'Hallo': 'Hello', 'Guten Tag': 'Good day', 'Wie geht es dir?': 'How are you?'}


### Apply code

In [10]:
print(os.getcwd())

os.chdir("../output")

c:\DATEN\PHD\Article_SoftRobotIntervention\Materials\translation process\automatize translation CAM data\data


In [14]:
save_interval = 10

dictionary_out = {}
counter = 0
# Loop through the list in chunks
chunk_size = 20
for i in range(0, len(df_text), chunk_size):
    # Select the next XX words (or fewer, if there are less than XX remaining)
    counter += 1
    chunk = df_text[i:i + chunk_size]
    
    result = basic_API_call(prompt=prompt_template,
    language="German",
    translate=chunk,
    openai_api_key=key.openai_api_key,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000)
    
    # Extract the 'parsed' section from the JSON data
    parsed_section = result.get('parsed', {})
    dictionary = parsed_section.get('dictionary')
    
    # Convert the string back to a Python dictionary
    #print(type(dictionary)) 
    dictionary = eval(dictionary)

    #print(len(dictionary.keys()))
    #print(dictionary) 

    if i == 0:
        dictionary_out = dictionary
    else:
        dictionary_out.update(dictionary)
        
        
    if counter % save_interval == 0:
        df_temp = pd.DataFrame(list(dictionary_out.items()), columns=['German', 'English'])
        df_temp.to_excel(f'translations_words_up_to_{counter}.xlsx', index=False)
        
        

# Save the DataFrame to an Excel file
df_final = pd.DataFrame(list(dictionary_out.items()), columns=['German', 'English'])

df_final.to_excel('translations_words.xlsx', index=False)
print("Translations saved to translations_words.xlsx")

c:\DATEN\PHD\Article_SoftRobotIntervention\Materials\translation process\automatize translation CAM data\data
Tokens Used: 613
	Prompt Tokens: 358
	Completion Tokens: 255
Successful Requests: 1
Total Cost (USD): $0.005615
Total Tokens: 613
Prompt Tokens: 358
Completion Tokens: 255
Total Cost (USD): $0.005615
Tokens Used: 586
	Prompt Tokens: 346
	Completion Tokens: 240
Successful Requests: 1
Total Cost (USD): $0.00533
Total Tokens: 586
Prompt Tokens: 346
Completion Tokens: 240
Total Cost (USD): $0.00533
Tokens Used: 588
	Prompt Tokens: 345
	Completion Tokens: 243
Successful Requests: 1
Total Cost (USD): $0.00537
Total Tokens: 588
Prompt Tokens: 345
Completion Tokens: 243
Total Cost (USD): $0.00537
Tokens Used: 628
	Prompt Tokens: 380
	Completion Tokens: 248
Successful Requests: 1
Total Cost (USD): $0.00562
Total Tokens: 628
Prompt Tokens: 380
Completion Tokens: 248
Total Cost (USD): $0.00562
Tokens Used: 568
	Prompt Tokens: 341
	Completion Tokens: 227
Successful Requests: 1
Total Cost (

In [15]:
print(counter)

NameError: name 'counter' is not defined

## Translate written comments

In [12]:
print(os.getcwd())

c:\DATEN\PHD\Article_SoftRobotIntervention\Materials\translation process\automatize translation CAM data\output


In [65]:
print(len(df_comment))

1264


In [66]:
# Step 1: Set up the OpenAI API
client = OpenAI(
    api_key=key.openai_api_key,
)

# Define the array of German words
comments_to_translate = df_comment['comment']
comments_ids_to_translate = df_comment['id']

# Function to translate a word using ChatGPT API
def translate_comment(comment):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that translates German comments to English."},
                {"role": "user", "content": f'Translate the following German comment to English: "{comment}" Be careful with spelling errors and do not use capital letters for adjectives. Only return the translation in quotes.'}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating comment '{comment}': {e}")
        return None

# Step 2: Translate the words with logging and saving
translations = []
counter = 0
save_interval = 50
print_interval = 100

for comment in comments_to_translate:
    translation = translate_comment(comment)
    if translation:
        translations.append(translation)
    counter += 1
    if counter % print_interval == 0:
        print(f"Current counter: {counter}")
    if counter % save_interval == 0:
        df_temp = pd.DataFrame({'ID': comments_ids_to_translate[:counter], 'German': comments_to_translate[:counter], 'English': translations})
        df_temp.to_excel(f'translations_comments_up_to_{counter}.xlsx', index=False)

# Final save
df_translated_comments = pd.DataFrame({
    'ID': comments_ids_to_translate,
    'German': comments_to_translate,
    'English': translations
})
df_translated_comments.to_excel('translations_comments.xlsx', index=False)

print(df_translated_comments)


Current counter: 100
Current counter: 200
Current counter: 300
Current counter: 400
Current counter: 500
Current counter: 600
Current counter: 700
Current counter: 800
Current counter: 900
Current counter: 1000
Current counter: 1100
Current counter: 1200
                                        ID  \
21    ac288548-733b-4da7-8556-1c181705a62c   
26    aa22a45d-df3c-4b06-972f-d19396c2f3fc   
28    f54edbb4-882a-42ed-9ebb-f2b06d3bcdda   
29    4c37e5b7-ceb9-470c-a545-c13847bfdb37   
36    ffe2196d-b660-4860-a342-f76d7cf7b183   
...                                    ...   
6357  da266df9-20a0-4caa-8d6c-8c7f7b5be9d5   
6358  11ccf4ec-dc3e-4641-984a-fd20a0e984fd   
6359  b2277901-9040-4914-a750-a628eb13cb83   
6405  cc3488fd-55d5-4da2-aa46-424b82d6f9a7   
6406  efd03640-89cb-4566-a2dc-24c02af6f2df   

                                                 German  \
21                                           Macht Spaß   
26    wahrscheinlich kommt nur noch der Roboter und ...   
28             